In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import faiss

In [2]:
train_data = pd.read_csv('fraudTrain.csv',index_col=0)
# pd.set_option('display.max_columns', None)
test_data = pd.read_csv('fraudTest.csv',index_col=0)

In [3]:
train_data['trans_date_trans_time']=pd.to_datetime(train_data['trans_date_trans_time'])
test_data['trans_date_trans_time']=pd.to_datetime(test_data['trans_date_trans_time'])

In [4]:
train_data['year'] = train_data['trans_date_trans_time'].dt.year
train_data['month'] = train_data['trans_date_trans_time'].dt.month
train_data['day'] = train_data['trans_date_trans_time'].dt.day
train_data['hour'] = train_data['trans_date_trans_time'].dt.hour
test_data['year'] = test_data['trans_date_trans_time'].dt.year
test_data['month'] = test_data['trans_date_trans_time'].dt.month
test_data['day'] = test_data['trans_date_trans_time'].dt.day
test_data['hour'] = test_data['trans_date_trans_time'].dt.hour

In [5]:
train_data.drop('trans_date_trans_time',axis=1,inplace=True)
test_data.drop('trans_date_trans_time',axis=1,inplace=True)

In [6]:
train_data['dob'] = pd.to_datetime(train_data['dob'])
test_data['dob'] = pd.to_datetime(test_data['dob'])

In [7]:
from datetime import date
def convert_to_age(dob):
    today = date.today()
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

train_data['dob'] = train_data['dob'].apply(convert_to_age)
test_data['dob']=test_data['dob'].apply(convert_to_age)

In [8]:
train_data.rename(columns={'dob':'age'},inplace=True)
columns_to_drop = ['first','last','gender','job']
train_data.drop(columns=columns_to_drop,axis=1,inplace=True)

test_data.rename(columns={'dob':'age'},inplace=True)
columns_to_drop = ['first','last','gender','job']
test_data.drop(columns=columns_to_drop,axis=1,inplace=True)

In [9]:
columns_to_drop = ['cc_num','trans_num','street','city','state','zip']
train_data.drop(columns=columns_to_drop,inplace=True)
test_data.drop(columns=columns_to_drop,inplace=True)

In [10]:
train_data.to_numpy()
test_data.to_numpy()
train_data

,merchant,category,amt,lat,long,city_pop,age,unix_time,merch_lat,merch_long,is_fraud,year,month,day,hour
0,"fraud_Rippin, Kub and Mann",misc_net,4.97,36.0788,-81.1781,3495,37,1325376018,36.011293,-82.048315,0,2019,1,1,0
1,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,48.8878,-118.2105,149,46,1325376044,49.159047,-118.186462,0,2019,1,1,0
2,fraud_Lind-Buckridge,entertainment,220.11,42.1808,-112.2620,4154,63,1325376051,43.150704,-112.154481,0,2019,1,1,0
3,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,46.2306,-112.1138,1939,58,1325376076,47.034331,-112.561071,0,2019,1,1,0
4,fraud_Keeling-Crist,misc_pos,41.96,38.4207,-79.4629,99,39,1325376186,38.674999,-78.632459,0,2019,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,fraud_Reichel Inc,entertainment,15.56,37.7175,-112.4777,258,63,1371816728,36.841266,-111.690765,0,2020,6,21,12
1296671,fraud_Abernathy and Sons,food_dining,51.70,39.2667,-77.5101,100,45,1371816739,38.906881,-78.246528,0,2020,6,21,12
1296672,fraud_Stiedemann Ltd,food_dining,105.93,32.9396,-105.8189,899,57,1371816752,33.619513,-105.130529,0,2020,6,21,12
1296673,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,43.3526,-102.5411,1126,44,1371816816,42.788940,-103.241160,0,2020,6,21,12


In [11]:
X_train = train_data.drop(columns='is_fraud')
y_train = train_data['is_fraud']
X_test = test_data.drop(columns='is_fraud')
y_test = test_data['is_fraud']

In [12]:
numerical_cols =  X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), [0, 1])], remainder='passthrough')
X_train_encoded = ct.fit_transform(X_train[['merchant', 'category']])

NameError: name 'X_train' is not defined

In [ ]:
X_test_encoded = ct.transform(X_test[['merchant', 'category']])

In [ ]:
encoded_X_train = pd.DataFrame(X_train_encoded, columns=ct.get_feature_names_out(), index=X_train.index).to_numpy()
encoded_X_test = pd.DataFrame(X_test_encoded, columns=ct.get_feature_names_out(), index=X_test.index).to_numpy()

In [ ]:
X_train = X_train.drop(columns=['merchant', 'category'])
X_test = X_test.drop(columns=['merchant', 'category'])
X_train = np.hstack((X_train, encoded_X_train))
X_test = np.hstack((X_test, encoded_X_test))

In [ ]:
# filename = 'X_train.dat'
# X_train_memmap = np.memmap(filename, dtype=np.float16, mode='w+', shape=(1296675, 719))
# X_train_memmap[:] = X_train[:]  # Save data to disk
# X_train_memmap.flush()  # Write changes to disk


In [ ]:
# import numpy as np
# import faiss
# import os

# # Define index file path
# index_path = "faiss_index.idx"

# # Number of clusters (tune this based on dataset size)
# nlist = 100  

# if not os.path.exists(index_path):  # Check if index already exists
#     print("Creating and saving FAISS index...")

#     # Create FAISS index with clustering
#     quantizer = faiss.IndexFlatL2(X_train.shape[1])  # Base index
#     index = faiss.IndexIVFFlat(quantizer, X_train.shape[1], nlist, faiss.METRIC_L2)

#     # Convert X_train to float32 and train index
#     X_train_fp32 = X_train.astype(np.float32)
#     index.train(X_train_fp32)  # Train clustering structure
#     index.add(X_train_fp32)  # Add training data

#     # Save index to disk
#     faiss.write_index(index, index_path)
# else:
#     print("Loading FAISS index from disk...")
#     index = faiss.read_index(index_path)  # Load index from disk

# # Set number of clusters to search for better speed-memory tradeoff
# index.nprobe = 10  

# # Reduce X_test batch size to prevent memory issues
# batch_size = 10000  
# y_pred_list = []

# for i in range(0, X_test.shape[0], batch_size):
#     batch = X_test[i:i + batch_size].astype(np.float32)
#     _, y_pred_batch = index.search(batch, k=5)
#     y_pred_list.append(y_pred_batch)

# y_pred = np.vstack(y_pred_list)  # Combine results

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree', n_jobs=-1)
classifier.fit(X_train,y_train)

In [ ]:
X_sample = X_test[:100]
y_sample_pred = classifier.predict(X_sample)
y_sample_pred

In [ ]:
batch_size = 500
num_batches = int(np.ceil(len(X_test) / batch_size))
y_pred_list = []
for i in range(num_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(X_test))
    print(f'Processing Batch {i+1}/{num_batches}')
    batch_pred = classifier.predict(X_test[start:end])
    y_pred_list.append(batch_pred)
y_pred = np.concatenate(y_pred_list)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm
from sklearn.metrics import accuracy_score
ac = accuracy_score(y_test,y_pred)
ac